In [20]:
from glob import glob
from tqdm.auto import tqdm
from univread import read as imread
import os
import numpy as np
import tifffile

In [5]:
examples = glob('/mnt/HD-LSDF/Xenopus/2021_11_Xenopus/test_janes/Xenopus_*_v2')

In [19]:
names = ['1', '4', '8', '12', '16']

In [21]:
for foldername in tqdm(examples):
    flats = imread(foldername+'/flats/')
    darks = imread(foldername+'/darks/')
    radios = imread(foldername+'/radios/')
    
    flats_r = flats.reshape((5, 200, 2016, 2016))
    darks_r = darks.reshape((5, 200, 2016, 2016))
    radios_r = np.moveaxis(radios.reshape((200, 5, 2016, 2016)), 1, 0)
    
    for i in tqdm(range(5), leave=False):
        os.makedirs(foldername+f'/{names[i]}/flats')
        os.makedirs(foldername+f'/{names[i]}/darks')
        os.makedirs(foldername+f'/{names[i]}/radios')
        
        tifffile.imsave(foldername+f'/{names[i]}/flats/0.tif', flats_r[i])
        tifffile.imsave(foldername+f'/{names[i]}/darks/0.tif', darks_r[i])
        tifffile.imsave(foldername+f'/{names[i]}/radios/0.tif', radios_r[i])

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
for folder_name in tqdm(examples):
    for sampling_name in tqdm(names, leave=False):
        os.system(f'cd {folder_name}/{sampling_name}; tofu flatcorrect --darks darks/0.tif --flats flats/0.tif  --projections radios/0.tif --fix-nan-and-inf --absorptivity --output radios_corrected/0.tif')

In [22]:
from skimage.morphology import disk
from skimage.exposure import rescale_intensity

def get_random_voxels(img, count):
    return img[tuple([np.random.randint(0, img.shape[i], count) for i in range(img.ndim)])]

CHUNK_SIZE = 256
import dask.array as da
from skimage.exposure import rescale_intensity
from functools import partial

def scale_intensity_parallel(img, in_percentiles=(0.1, 99.9), out_range=(-1, 1)):
    rs = get_random_voxels(img, 1_000_000)
    f = np.percentile(rs, in_percentiles[0])
    t = np.percentile(rs, in_percentiles[1])
    
    img = da.from_array(img, chunks=CHUNK_SIZE)
    scaler = partial(rescale_intensity, in_range=(f,t), out_range=out_range)
    img = img.map_blocks(scaler, dtype=img.dtype)
    img = np.array(img, dtype=img.dtype)

    return img

def get_scaled_volume(v):
    v_f = v.astype(np.float32)
    return scale_intensity_parallel(v_f)

In [23]:
for folder_name in tqdm(examples):
    for sampling_name in tqdm(names, leave=False):
        current_image = tifffile.imread(os.path.join(folder_name, sampling_name, 'radios_corrected', '0.tif'))
        tifffile.imsave(os.path.join(folder_name, sampling_name, 'radios_corrected', 'scaled_0.tif'), get_scaled_volume(current_image))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]